# Take Home Assignment Clean and Transform data

Assumption (fyi: This solution is not the best time complexity. It can be optimize for speed up. This solution is focus on logic)
1. ISO format YYYY-MM-DD in A.D year 
2. assume input is csv of 4 column
3. assume there is no null column
4. assume date input format is d/m/y
5. assume correct phone number is length of 9 and 10
6. if data cannot transform (invalid data) move it in to invalid date frame
7. transform mask phone and email
8. data in invalid csv may be change

data rule : correct email, phone, date
if data match all rule that row is correct row move to cleaned data if not move to invalid data
we only transform and clean correct date don't care about incorrect data if there were transformed

## Import Library

In [1]:
import pandas as pd
from datetime import date
import numpy as np
import re

## Read CSV

In [2]:
df = pd.read_csv('input.csv', names=['date','name', 'email','phone'])
df

,date,name,email,phone
0,1/19/2566,ชื่นใจ สวยสกุล,churn.jai.s@gmail.com,090-55508888
1,4/20/2566,สมชาย เด็ดเดี่ยว,lnwza777@hotmail.com,0815678900
2,21/22/2566,สมุย เด็ดเดี่ยว,samurai@gmail.com,1112
3,2/30/2563,จอห์น มองอะไร,bkk888@gmail.com,099-668-5555
4,1/19/2566,ชื่นใจ สวยสกุล,churn.jai.s@gmail.com,090-55508888
5,4/5/2566,สมชาย เด็ดเดี่ยว,lnwza777@hotmail.com,0815678900
6,21/12/2566,สมุย เด็ดเดี่ยว,samurai@gmail.com,1112
7,2/10/2563,จอห์น มองอะไร,bkk888@gmail.com,099-668-5555
8,1/19/2566,ชื่นใจ สวยสกุล,churn.jai.sasdfasdf,1111111
9,4/20/2566,สมชาย เด็ดเดี่ยว,lnwza777@hotmail.com,0815678900


## Transform Data

split date to day month year

In [3]:
# df['day'] = df['date'].str.split('/').apply(lambda x: x[0].strip())
# df['month'] = df['date'].str.split('/').apply(lambda x: x[1].strip())
# df['year'] = df['date'].str.split('/').apply(lambda x: int(x[2].strip()) - 543)

df[['day', 'month', 'year']] = df['date'].str.split('/', expand=True)
df['day'] = df['day'].str.strip().str.zfill(2)
df['month'] = df['month'].str.strip().str.zfill(2)
df['year'] = df['year'].str.strip().astype(int) - 543

create iso date

In [4]:
df['format_date'] = df['year'].astype(str) + '-' + df['month'] + '-' + df['day']

split name to first name and last name (assume firstname and lastname is seperate by space)

In [5]:
df[['first_name', 'last_name']] = df['name'].str.split(' ', expand=True)

clean phone number format

In [6]:
df['phone'] = df['phone'].str.replace('[^\d]+', '', regex=True)

Old function did not used.

In [7]:
# not faster
def check_email(email: str):
  regex = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
  if not regex.match(email):
    return False
  return True

In [8]:
def check_date(date: str):
  # assume input in date column is number/number/number
  # valid date is correct format of date don't care the input is in the future
  # year, month, day =  format_date.split('/')
  day, month, year =  date.split('/')
  year = int(year.strip()) - 543
  month = int(month.strip())
  day = int(day.strip())

  day_count_for_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

  if year%4==0 and (year%100 != 0 or year%400==0):
        day_count_for_month[2] = 29
  
  if (1 <= month <= 12 and 1 <= day <= day_count_for_month[month]):
    return True
  return False

mask valid date column

In [9]:
# df['valid date'] = df['date'].apply(check_date)
date_series = pd.to_datetime(df['format_date'], errors='coerce')
df['is_valid_date'] = ~date_series.isna()

mask valid email column

In [10]:
# df['valid email'] = df['email'].apply(check_email) #old solution
email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
df['is_valid_email'] = df['email'].str.contains(email_pattern)

mask valid phone column

In [11]:
df['is_valid_phone'] = (df['phone'].str.len() >= 9) & (df['phone'].str.len() <= 10)

masking phone number (valid phone number is phone number length 9 or 10)

In [12]:
df.loc[df['phone'].str.len() == 9,'phone'] = df['phone'].str.replace(r'^(\d{3})(\d{4})(\d{2})$', r'\1' + r'x'*4 + r'\3', regex=True)
df.loc[df['phone'].str.len() == 10,'phone'] = df['phone'].str.replace(r'^(\d{3})(\d{5})(\d{2})$', r'\1' + r'x'*5 + r'\3', regex=True)

mask email 

option 1: dynamic *

In [13]:
# df[['username', 'domain']] = df['email'].str.split('@', expand=True)
# df.fillna("",inplace=True)
# len_user_name = (df['username'].str.len()-2).to_numpy()
# star = np.array(['*' for e in len_user_name])
# mask_part = np.char.multiply(star, len_user_name)
# df['email'] = df['username'].str.slice(start=0, stop=1) + mask_part + df['username'].str.slice(-1) + '@' + df['domain']

option 2: fix *

In [14]:
df['email'] = df['email'].replace(r'(.{1}).*(.{1}@.+\..+)', r'\1****\2', regex=True)

drop unused column

In [15]:
df.drop(['name','date','day','month','year'], inplace=True, axis=1)

split to valid and invalid df and drop unused column

In [16]:
valid_rows = df[df['is_valid_date'] & df['is_valid_email'] & df['is_valid_phone']]
valid_df = valid_rows.copy()
valid_df.drop(['is_valid_date','is_valid_email','is_valid_phone'], axis=1, inplace=True)

In [17]:
invalid_rows = df[~(df['is_valid_date'] & df['is_valid_email'] & df['is_valid_phone'])]
invalid_df = invalid_rows.copy()
invalid_df.drop(['is_valid_date','is_valid_email','is_valid_phone'], axis=1, inplace=True)

In [18]:
invalid_df

,email,phone,format_date,first_name,last_name
0,c****s@gmail.com,09055508888,2023-19-01,ชื่นใจ,สวยสกุล
1,l****7@hotmail.com,081xxxxx00,2023-20-04,สมชาย,เด็ดเดี่ยว
2,s****i@gmail.com,1112,2023-22-21,สมุย,เด็ดเดี่ยว
3,b****8@gmail.com,099xxxxx55,2020-30-02,จอห์น,มองอะไร
4,c****s@gmail.com,09055508888,2023-19-01,ชื่นใจ,สวยสกุล
6,s****i@gmail.com,1112,2023-12-21,สมุย,เด็ดเดี่ยว
8,churn.jai.sasdfasdf,1111111,2023-19-01,ชื่นใจ,สวยสกุล
9,l****7@hotmail.com,081xxxxx00,2023-20-04,สมชาย,เด็ดเดี่ยว
10,s****i@gmail.com,1112,2023-22-21,สมุย,เด็ดเดี่ยว
11,b****8@gmail.com,009xxxx50,2020-30-02,จอห์น,มองอะไร


In [19]:
valid_df

,email,phone,format_date,first_name,last_name
5,l****7@hotmail.com,081xxxxx00,2023-05-04,สมชาย,เด็ดเดี่ยว
7,b****8@gmail.com,099xxxxx55,2020-10-02,จอห์น,มองอะไร
13,b****f@gmail.com,089xxxxx22,2030-12-01,จอห์น,มองอะไร


## save data to csv

In [20]:
valid_df.to_csv('cleaned_data.csv', columns = ['format_date', 'first_name', 'last_name', 'email', 'phone'], index=False)

In [21]:
invalid_df.to_csv('invalid_data.csv', columns = ['format_date', 'first_name', 'last_name', 'email', 'phone'], index=False)